In [5]:
expect_result_path = "expectResult.txt"
cps_path = "cps.txt"
resultados = "resultados.txt"

# Open the files
with open(expect_result_path, "r") as f:
    expect_result = f.read()
with open(cps_path, "r") as f:
    cps = f.read()
with open(resultados, "r") as f:
    resultados = f.read()

expect_result = expect_result.splitlines()
cps = cps.splitlines()
resultados = resultados.splitlines()



In [ ]:
from openai import OpenAI

Visma_Seek_Api = "sk-8811953733e4417abf2e0b41c56eb5a9"

def get_feedback_on_corrections(expect_result, resultados, batch_size=10):
    client = OpenAI(api_key=Visma_Seek_Api, base_url="https://api.deepseek.com")

    # Dividir los datos en lotes
    batches = [
        (expect_result[i:i + batch_size], resultados[i:i + batch_size])
        for i in range(0, len(expect_result), batch_size)
    ]

    feedback = []
    for batch_expect, batch_result in batches:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "Eres un experto en pruebas de software. Tu tarea es analizar las correcciones realizadas al 'expect_result' y generar observaciones sobre las diferencias."},
                {"role": "user", "content": f"Resultados esperados originales: {batch_expect}"},
                {"role": "user", "content": f"Resultados corregidos: {batch_result}"},
                {"role": "user", "content": "Genera observaciones sobre las diferencias entre los resultados esperados originales y los corregidos. Usa el formato: OBS 1: Observación sobre la diferencia, EXPECTED: xxxxxxx, RESULT: xxxxxxx. No incluyas comillas ni formato adicional."}
            ],
            stream=False
        )
        # Acumular el feedback
        feedback.append(response.choices[0].message.content)
        break

    # Combinar todas las observaciones en un resumen
    return "\n".join(feedback)

# Llamar a la función y generar el feedback
feedback = get_feedback_on_corrections(expect_result, resultados)
print("Feedback sobre las correcciones:")
print(feedback)

# Guardar el feedback en un archivo
with open("feedback_correcciones.txt", "w") as f:
    f.write(feedback)